# Practical ODE solvers

## Error estimation
We'd like to control the error in the solution:

$$E = |U^n - u(t_n)|.$$

Controlling the **global error** is very difficult and requires solving the problem over the whole time interval multiple times.

Instead, what's usually done is to control the **local (one-step) error** ${\mathcal L}^n$.  Since we don't have access to the exact solution, we can use two numerical solutions to do this.

At step $n$, suppose we start with $U^n$ and compute two new solutions $U^{n+1}, \hat{U}^{n+1}$, with methods of order $p$ and $p-1$ respectively.  Then the one-step errors for these solutions are

\begin{align*}
    {\mathcal L}^{n+1} = U^{n+1}-u(t_{n+1}) \approx C_1 k^{p+1} \\
    \hat{\mathcal L}^{n+1} = \hat{U}^{n+1}-u(t_{n+1}) \approx C_2 k^{p}.
\end{align*}

So if we take their difference, we get

$$
\|{\mathcal L}^{n+1} - \hat{{\mathcal L}}^{n+1}\| = \|U^{n+1}-u(t_{n+1}) - (\hat{U}^{n+1}-u(t_{n+1})\| \approx
  \|C_1k^{p+1} - C_2 k^{p}\| \approx \|C_2 k^p\|
$$

as long as $k$ is small.

Thus the **difference between the two solutions** gives an approximation of the **error in the lower-order method**.  So we could continue stepping forward using the low-order solution $\hat{U}^{n+1}$ and we'd have an approximation of the local error at each step.

However, it makes more sense to continue forward with the **high order solution**, for which the error is presumably even smaller.  This is called **local extrapolation**.

## Step size control

Let's say we want the one-step error at each step to be smaller than $\epsilon$:

$${\mathcal L}^{n+1} < \epsilon$$

We can control the error by adjusting the step size $k$.  In fact we want to choose a step size that yields an error close to $\epsilon$, since using a smaller timestep would be wasteful (doing "too much work" to get a needlessly accurate solution).

Thus we want to choose a step size $k_*$ such that

$$C_2 k_*^p \approx \epsilon.$$

Suppose our computed error estimate value is $\hat{\epsilon}\approx C_2 k^p$.  Thus we have

$$
\left(\frac{k_*}{k}\right)^{p} \approx \frac{\epsilon}{\hat{\epsilon}}
$$

which means

$$
k_* = k \left(\frac{\epsilon}{\hat{\epsilon}}\right)^{1/p},
$$

so we should modify the timestep by a factor $\left(\frac{\epsilon}{\hat{\epsilon}}\right)^{1/p}.$

A full step size control algorithm looks like this:

 - Take a step and compute the approximate error $\hat{\epsilon}$
 - If $\hat{\epsilon}>\epsilon$, reject the new solution value and go back to $t_n$.
 - If $\hat{\epsilon}\le \epsilon$, accept the new solution value and advance to $t_{n+1}$.
 - Either way, set the new step size to $k_* = \alpha k \left(\frac{\epsilon}{\hat{\epsilon}}\right)^{1/p}$
 
Here $\alpha$ is a safety factor that is a little smaller than 1.

More complicated approaches exist, based on optimal control theory and filtering methods from signal processing.  We won't go into the details of those.

## Embedded Runge-Kutta pairs

Getting an error estimate is nice, but having to compute two numerical solutions is expensive.  With Runge-Kutta methods, we can usually use the information we already have to get a second numerical approximation for free.

Recall that a Runge-Kutta method takes the form

\begin{align*}
Y_i & = U^n + k\sum_{j=1}^r a_{ij} f(Y_j,t_n+kc_j) & i=1,2,\dots,r \\
U^{n+1} & = U^n + k \sum_{j=1}^r b_j f(Y_j, t_n+kc_j).
\end{align*}

If we use a second method of the form

\begin{align*}
Y_i & = U^n + k\sum_{j=1}^r a_{ij} f(Y_j,t_n+kc_j) & i=1,2,\dots,r \\
U^{n+1} & = U^n + k \sum_{j=1}^r \hat{b}_j f(Y_j, t_n+kc_j)
\end{align*}

where the weights $a_{ij}$ for both methods are **the same** and only the weights $\hat{b}_j$ are different, then the second method can be computed essentially **for free**.

We call this an **embedded Runge-Kutta pair**.  We have to choose the second set of weights $\hat{b}_j$ so that the second method is of a lower order and gives a good error estimate.  Here's an example:

In [8]:
from nodepy import rk
rkm = rk.loadRKM('BS3')
print(rkm)

Bogacki-Shampine RK3(2)4

 0   |
 1/2 | 1/2
 3/4 |      3/4
 1   | 2/9  1/3  4/9
_____|____________________
     | 2/9  1/3  4/9  0
     | 7/24 1/4  1/3  1/8


In [10]:
print(rkm)

Bogacki-Shampine RK3(2)4

 0   |
 1/2 | 1/2
 3/4 |      3/4
 1   | 2/9  1/3  4/9
_____|____________________
     | 2/9  1/3  4/9  0
     | 7/24 1/4  1/3  1/8


This method has another useful property.  Notice that the last row of $A$ is equal to the weights $b$ of one method.  That means that stage $Y_3$ is also the new solution $U^{n+1}$.  When we take the next step, we'll need to evaluate $f(U^{n+1})$ anyway, so we can use $f(Y_3)$ "for free".

In other words, even though this looks like a 4-stage method, we only need to do 3 derivative evaluations per step.

## Dense output

If we use a high-order-accurate method and the solution is smooth, we can take **very big steps**.  But sometimes we need output from the simulation at shorter intervals.

It would be wasteful to use a small timestep just so that we can generate output at closer-spaced times.  Instead, we can use a cheaper method to generate highly accurate values **in between** the time step points.

For linear multistep methods, because we need to store the last several steps anyway, we can simply use a high-order interpolating polynomial.  This polynomial can be evaluated at any point where we need output, and (since the step values are all high order), the result will be high order accurate.

For Runge-Kutta methods, normally we only need to keep the most recent step in memory.  If we wanted to interpolate using just the step values $U^n, U^{n-1}, \dots$ then we would need to keep several past steps in memory, which is costly and inconvenient.

Instead, we could try to interpolate using the **stage values**.  For instance, using the fourth order method

\begin{align*}
Y_1 & = U^n \\
Y_2 & = U^n +\frac{k}{2}f(t_n,Y_1) \\
Y_3 & = U^n +\frac{k}{2}f(t_n+k/2,Y_2) \\
Y_4 & = U^n +kf(t_n+k/2,Y_3) \\
U^{n+1} & = U^n + \frac{k}{6} \left(f(t_n,Y_1) + 2f(t_n+k/2,Y_2) + 2f(t_n+k/2,Y_3) + f(t_n+k,Y_4) \right)
\end{align*}

we have approximate values at the start, middle, and end of the step.  Can we use these to get high-order output in between?

The simple answer is **no**, at least if we simply interpolate.  That's because the stage values at the middle point are **less accurate**.  For instance, $Y_2$ is based on an Euler step and is only a first-order approximation.

## Continuous Runge-Kutta methods

Instead, we can design special interpolation formulas that take into account the structure and accuracy of the Runge-Kutta stages.  Our Runge-Kutta method reads

\begin{align*}
Y_i & = U^n + k\sum_{j=1}^r a_{ij} f(Y_j,t_n+kc_j) & i=1,2,\dots,r \\
U^{n+1} & = U^n + k \sum_{j=1}^r b_j f(Y_j, t_n+kc_j).
\end{align*}

The idea is to create an interpolant of the form

$$
U^{n+\theta} \approx u(t_n + \theta k) \approx U^n + k \sum_{j=1}^r b_j(\theta) f(Y_j),
$$

where $0\le \theta \le 1$.  By carefully choosing the weight functions $b(\theta)$, we can get high order
accuracy in this way, without any extra storage or any extra derivative evaluations.

Here's an example of a 3-stage, 3rd-order method with a 2nd-order interpolation formula:

\begin{align*}
    Y_1 & = U^n \\
    Y_2 & = U^n + kf(Y_1) \\
    Y_3 & = U^n + \frac{k}{4}f(Y_1) + \frac{k}{4}f(Y_2) \\
    U^{n+1} & = U^n + k(f(Y_1) + f(Y_2) + 4 f(Y_3))/6 \\
    U^{n+\theta} & = U^n + (2\theta-\theta^2)f(Y_1) + (\theta^2-2\theta)f(Y_2) + \theta f(Y_3)
\end{align*}

For a method of order $p$, it is usually acceptable to use an interpolant of order $p-1$.  This is because the global error is one order worse than the local error and we won't use the interpolated values to continue stepping.

## Automatic adaptation of the order

Should we use a high-order or low-order method?

It depends on the accuracy we require and the smoothness of our solution.  Instead of trying to figure this out for each problem, we can design an algorithm that tries to figure it out for us, and chooses an appropriate high or low order method automatically.

We could even change the order of the method being used during the computation, depending on the local smoothness of the solution.  There are several available codes that do this.

## ODE solver libraries

Nowadays many well-designed solvers with error estimation, step size control, dense output, and other features are widely available through packages in MATLAB, Python, Mathematica, and other languages.  Many of these are based on Fortran or C codes written in the late 20th century.

In Python, the most standard library for solving ODEs is `scipy.integrate.solve_ivp`.  Let's look at the solvers included there.  You can learn more about them by reading the help page:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html

and the papers cited there.

### Explicit SciPy solvers

- `RK45`: A 7-stage embedded pair of orders 5 and 4, developed by Prince & Dormand.  Used with local extrapolation and a 4th-order continuous interpolant.  Because of the FSAL property, the cost is like that of a 6-stage method.
- `RK23`: An embedded pair of orders 3 and 2, developed by Bogacki & Shampine.  Used with local extrapolation and 3rd-order continuous interpolant based on Hermite interpolation.
- `DOP853`: An 8th-order method of Dormand & Prince with 12 stages, combined with two embedded error estimators of order 5 and 3, and a dense output interpolant.

Higher-order methods cost more per step (since they use more stages) but allow larger steps as long as the solution is smooth.  For very small error tolerances, DOP853 will be most efficient, while at very loose tolerances RK23 will be most efficient.  RK45 is recommended as a good method to try first.

### Implicit SciPy solvers
- `BDF`: Uses "Numerical differentiation methods" of order 1-5.  These are a small modification of the BDF methods, to get better stability regions.  Automatically adapts the order.  Uses a Newton method to solve for each step.  Reuses the Jacobian from a previous step until this causes slow convergence. 
- `Radau`: A 5th-order fully-implicit Runge-Kutta method with a 3rd-order embedded error estimator and 3rd-order interpolant.
- `LSODA`: This one tries to figure out for you if the problem is stiff, and choose an appropriate method.  Also adapts the order and step size automatically.  Uses Adams-Bashforth methods of order 1-12 for nonstiff problems and BDF methods of order 1-5 for stiff problems.  Due to Petzold & Hindmarsh.

### Explicit MATLAB solvers
- `ode23`: Same as SciPy's `RK23`
- `ode45`: Same as SciPy's `RK45`
- `ode113`: A group of linear multistep methods of order 1-13.  Starts with order 1 method (forward Euler) and then increases the order gradually based on measurement of the error.  Uses a predictor-corrector approach, combining Adams-Bashforth and Adams-Moulton methods.  It can be the best option of the problem is *mildly stiff* and evaluating $f$ is very expensive.

### Implicit MATLAB solvers
- `ode15s`: like SciPy's `BDF`
- `ode23s`: Rosenbrock method.  Like implicit Runge-Kutta, but only performs one Newton iteration (instead of iterating to convergence).  2nd-order method is L-stable; 3rd-order method is A-stable and used for error estimation.  Has a 2nd-order interpolant.